**Digit Recognizer with Binary and Multiclass Classification**





In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import Counter
from sklearn.metrics import confusion_matrix
import itertools
import seaborn as sns
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
%matplotlib inline

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
import keras

In [ ]:
train = pd.read_csv("../input/train.csv")
print(train.shape)
train.head()

X = train[train['label'].isin([0, 1])]


Y = train[train['label'].isin([0, 1])]['label']


X = X.drop(['label'], axis = 1)


In [ ]:
z_train = Counter(train['label'])
z_train

In [ ]:
sns.countplot(train['label'])

In [ ]:
#data girişi
test= pd.read_csv("../input/test.csv")
print(test.shape)
test.head()

In [ ]:
x_train = (train.ix[:,1:].values).astype('float32')
y_train = train.ix[:,0].values.astype('int32') 
x_test = test.values.astype('float32')

In [ ]:
%matplotlib inline

plt.figure(figsize=(12,10))
x, y = 10, 4
for i in range(40):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_train[i].reshape((28,28)),interpolation='nearest')
plt.show()

In [ ]:
#sigmoid fonksiyonu
def sigmoid(z):
    s = 1.0 / (1+ np.exp(-z))
    return s

In [ ]:
#sinir ağının kurulumu
def ag_mimarisi(X,Y):
    n_x = X.shape[0]
    n_h = 10
    n_y = Y.shape[0]

    return (n_x, n_h , n_y)

In [ ]:
#ag sartlarini belirleme
def ag_sartlarini_belirleme(n_x, n_h, n_y):
    w1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    w2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    return {"W1": w1, "b1":b1, "W2":w2, "b2":b2}

In [ ]:
#ileri yayılım
def ileri_yayilim(X, params):
    z1 = np.dot(params['W1'], X) + params ['b1']
    a1 = sigmoid(z1)
    
    z2 = np.dot(params['W2'], a1) + params ['b2']
    a2 = sigmoid(z2)
    
    return {"Z1": z1,"A1": a1, "Z2": z2, "A2":a2}

In [ ]:
#şimdiki değer ile tahmin edilen değerin hesaplama hatasının tespiti için maliyet fonksiyonu

def hesaplama_hatasi(tahmin, guncel):
    logprobs = np.multiply(np.log(tahmin), guncel) + np.multiply(np.log(1-tahmin) , 1-guncel)
    maliyet = -np.sum(logprobs) / guncel.shape[1]
    return np.squeeze(maliyet)

In [ ]:
#geri yayılım algoritması

def geri_yayilim(params, activations, X, Y):
    m = X.shape[1]
    
    #çıkış katmanı
    dz2 = activations['A2'] - Y
    dw2 = np.dot(dz2, activations['A1'].T) / m
    db2 = np.sum(dz2, axis = 1, keepdims = True) / m
    
    #gizli katman
    dz1 = np.dot(params['W2'].T, dz2)*(1-np.power(activations['A1'], 2))
    dw1 = np.dot(dz1, X.T)/m
    db1 = np.sum(dz1, axis = 1, keepdims= True)/m
    
    return {"dW1": dw1, "db1": db1, "dW2": dw2, "db2": db2}

def parametre_guncelleme(params, derivatives, alfa = 1.2): 
    
    params['W1'] = params['W1']- alfa * derivatives['dW1']
    params['b1'] = params['b1']- alfa * derivatives['db1']
    params['W2'] = params['W2']- alfa * derivatives['dW2']
    params['b2'] = params['b2']- alfa * derivatives['db2']
    return params

#alfa modelin öğrenme oranıdır..

**Modelin kurulumu ve eğitimi**

In [ ]:
def sinir_agi(X, Y, n_h, num_iterations = 100):
    n_x = ag_mimarisi(X, Y)[0]
    n_y = ag_mimarisi(X, Y)[2]
    
    params = ag_sartlarini_belirleme(n_x, n_h, n_y)
    for i in range(0, num_iterations):
        sonuclar = ileri_yayilim(X, params)
        hata = hesaplama_hatasi(sonuclar['A2'], Y)
        turev = geri_yayilim(params, sonuclar, X, Y)
        params = parametre_guncelleme(params, turev)
    return params

In [ ]:
y = Y.values.reshape(1, Y.size)
x = X.T.as_matrix()

model = sinir_agi(x, y, n_h = 10, num_iterations = 10)

In [ ]:
#Tahmin
def tahmin(parameters, X):
    sonuclar = ileri_yayilim(X, parameters)
    print (sonuclar['A2'][0])
    tahminler = np.around(sonuclar['A2'])
    return tahminler

tahminler = tahmin(model, x)
print('Doğruluk: %d' % float((np.dot(y, tahminler.T) + np.dot(1-y, 1- tahminler.T))/ float(y.size)*100) + '%')

**2. Çoklu Sınıflandırma - Digit Recognizer**

Önceki adımda, ikili sınıflandırma yapan bir NN in uygulanımını incelemeye çalıştım. Doğruluğu 95%. Şimdi çoklu sınıflandırmada, Python kütüphanesi olan sklearn kullanacağım.

In [ ]:
#kütüphaneleri alma
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn import metrics

Y = train['label'][:10000] 
X = train.drop(['label'], axis = 1)[:10000]
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.20, random_state = 42)

**Modelin 10 gizli katman ile eğitimi**

In [ ]:
model = neural_network.MLPClassifier(alpha = 1e-5, hidden_layer_sizes=(5,), solver = 'lbfgs', random_state=18)
model.fit(x_train, y_train)

**Tahmin**

In [ ]:
tahmin_edilen = model.predict(x_val)
print("Sınıflandırma:\n %s:" % (metrics.classification_report(y_val, tahmin_edilen)))

***Evrişimli Sinir Ağı ile Digit Recognizer***

In [ ]:
Y = train['label']
X = train.drop(['label'], axis=1)

x_train, x_val, y_train, y_val = train_test_split(X.as_matrix(), Y.as_matrix(), test_size = 0.10, random_state = 42)

In [ ]:
#ağ parametrelerini belirleme
batch_size = 128 #küme büyüklüğü
num_classes = 10 #toplam sınıflandırma sayısı, 0 dan 9 a
epochs = 5

#resim boyutları
img_rows, img_cols = 28, 28

In [ ]:
#Resim verilerini 4 boyutlu vektör haline getiriyoruz. Bir sonraki adım ise bu girdileri en yüksek piksel değeri olan 255 e bölerek normal hale getirmek.

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_val = x_val.astype('float32')
x_val /= 255

input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

Xtest = test.as_matrix()
Xtest = Xtest.reshape(Xtest.shape[0], img_rows, img_cols, 1)

**Evrişimli Sinir Ağı Mimarisini Kurmak - Digit Recognizer**

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])

model.summary()

***Modeli Eğitme***

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, verbose = 1, validation_data = (x_val, y_val))
dogruluk = model.evaluate(x_val, y_val, verbose = 0)
print('Test doğruluğu:', dogruluk[1])

In [ ]:
tahm = model.predict(Xtest)
y_classes = tahm.argmax(axis = -1)
res = pd.DataFrame()
res['ImageId'] = list(range(1,28001))
res['Label'] = y_classes
res.to_csv("sonuc.csv", index = False)